In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os
import urllib

import tensorflow as tf
tf.logging.set_verbosity(tf.logging.INFO)
import numpy as np

import matplotlib.pyplot as plt

IRIS_TRAINING = "iris_training.csv"
IRIS_TRAINING_URL = "http://download.tensorflow.org/data/iris_training.csv"

IRIS_TEST = "iris_test.csv"
IRIS_TEST_URL = "http://download.tensorflow.org/data/iris_test.csv"

%matplotlib inline

In [2]:
# Load datasets.
training_set = tf.contrib.learn.datasets.base.load_csv_with_header(
    filename=IRIS_TRAINING,
    target_dtype=np.int,
    features_dtype=np.float32)

test_set = tf.contrib.learn.datasets.base.load_csv_with_header(
    filename=IRIS_TEST,
    target_dtype=np.int,
    features_dtype=np.float32)

In [3]:
training_set.data.shape

(120, 4)

In [4]:
# Specify that all features have real-value data
feature_columns = [tf.feature_column.numeric_column("x", shape=[4])]

# Build 3 layer DNN with 10, 20, 10 units respectively.
classifier = tf.estimator.DNNClassifier(feature_columns=feature_columns,
                                        hidden_units=[10, 20, 10],
                                        n_classes=3,
                                        model_dir="/tmp/iris_model")

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_keep_checkpoint_max': 5, '_save_checkpoints_secs': 600, '_save_summary_steps': 100, '_model_dir': '/tmp/iris_model', '_tf_random_seed': 1, '_log_step_count_steps': 100, '_session_config': None}


In [5]:
tf.feature_column.numeric_column?

In [6]:
feature_columns

[_NumericColumn(key='x', shape=(4,), default_value=None, dtype=tf.float32, normalizer_fn=None)]

In [7]:
# tf.estimator.DNNClassifier?

In [8]:
# Define the training inputs
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": np.array(training_set.data)},
    y=np.array(training_set.target),
    num_epochs=None,
    shuffle=True)

In [9]:
# Train model.
classifier.train(input_fn=train_input_fn, steps=2000)

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from /tmp/iris_model/model.ckpt-4000
INFO:tensorflow:Saving checkpoints for 4001 into /tmp/iris_model/model.ckpt.
INFO:tensorflow:step = 4001, loss = 3.63822
INFO:tensorflow:global_step/sec: 754.432
INFO:tensorflow:step = 4101, loss = 3.1336 (0.135 sec)
INFO:tensorflow:global_step/sec: 791.916
INFO:tensorflow:step = 4201, loss = 7.32781 (0.126 sec)
INFO:tensorflow:global_step/sec: 776.808
INFO:tensorflow:step = 4301, loss = 3.6719 (0.129 sec)
INFO:tensorflow:global_step/sec: 770.387
INFO:tensorflow:step = 4401, loss = 9.65957 (0.130 sec)
INFO:tensorflow:global_step/sec: 752.043
INFO:tensorflow:step = 4501, loss = 8.03901 (0.133 sec)
INFO:tensorflow:global_step/sec: 742.865
INFO:tensorflow:step = 4601, loss = 5.72575 (0.135 sec)
INFO:tensorflow:global_step/sec: 777.273
INFO:tensorflow:step = 4701, loss = 4.12203 (0.129 sec)
INFO:tensorflow:global_step/sec: 747.725
INFO:tensorflow:step = 4801, loss = 1.87848

In [7]:
# Define the test inputs
test_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": np.array(test_set.data)},
    y=np.array(test_set.target),
    num_epochs=1,
    shuffle=False)

# Evaluate accuracy.
accuracy_score = classifier.evaluate(input_fn=test_input_fn)["accuracy"]

print("\nTest Accuracy: {0:f}\n".format(accuracy_score))

INFO:tensorflow:Starting evaluation at 2017-10-20-03:21:42
INFO:tensorflow:Restoring parameters from /tmp/iris_model/model.ckpt-2000
INFO:tensorflow:Finished evaluation at 2017-10-20-03:21:43
INFO:tensorflow:Saving dict for global step 2000: accuracy = 0.966667, average_loss = 0.0549388, global_step = 2000, loss = 1.64816

Test Accuracy: 0.966667



In [9]:
# Classify two new flower samples.
new_samples = np.array(
    [[6.4, 3.2, 4.5, 1.5],
     [5.8, 3.1, 5.0, 1.7]], dtype=np.float32)
predict_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": new_samples},
    num_epochs=1,
    shuffle=False)

predictions = list(classifier.predict(input_fn=predict_input_fn))
predicted_classes = [p["classes"] for p in predictions]

print(
    "New Samples, Class Predictions:    {}\n"
    .format(predicted_classes))

INFO:tensorflow:Restoring parameters from /tmp/iris_model/model.ckpt-2000
New Samples, Class Predictions:    [array([b'1'], dtype=object), array([b'2'], dtype=object)]



In [10]:
predictions

[{'class_ids': array([1]),
  'classes': array([b'1'], dtype=object),
  'logits': array([-5.3435173 ,  4.682055  , -4.30464888], dtype=float32),
  'probabilities': array([  4.42462006e-05,   9.99830723e-01,   1.25040460e-04], dtype=float32)},
 {'class_ids': array([2]),
  'classes': array([b'2'], dtype=object),
  'logits': array([-7.69936657,  0.39931303,  2.00755262], dtype=float32),
  'probabilities': array([  5.07047589e-05,   1.66824698e-01,   8.33124578e-01], dtype=float32)}]